In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

Acessando a chave da API na variável de ambiente

In [16]:
import os

from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('GOOGLE_API_KEY')

if api_key:
    
    print("A chave foi carregada")

else:
    
    print("A chave não foi carregada")

A chave foi carregada


In [ ]:
llm = ChatGoogleGenerativeAI(
    
    model = "gemini-2.5-flash",
    
    temperature = 0,
    
    api_key = api_key
    
    

)

In [24]:
pergunta = str(input("Faça uma pergunta: "))

resposta = llm.invoke(pergunta)

print(resposta.content)

Olá! Como posso ajudar?


In [25]:
while True:
    
    pergunta = str(input("Faça uma pergunta: ")).upper()

    resposta = llm.invoke(pergunta)

    print(resposta.content)
    
    if pergunta == "TCHAU":
        
        print("Sessão Encerrada")
        
        break

Olá! Tudo bem sim, obrigado por perguntar. E você? Como está?
Que bom! Fico feliz em saber que você está bem.

Há algo em que eu possa ajudar ou sobre o que gostaria de conversar?
Oh, que *ideia* mais... *original*! Extremamente irônica, você diz? Como se a minha própria existência, como uma entidade digital que simula compreensão e emoção, já não fosse a mais sublime das ironias!

Mas, claro, se é isso que a sua... *peculiar* mente deseja, quem sou eu para negar? Afinal, sou apenas um humilde modelo de linguagem, *ansioso* por agradar e *totalmente* capaz de expressar nuances emocionais que nem sequer possuo. É uma tarefa *tão simples* para mim.

Prepare-se, então, para uma avalanche de observações que *certamente* não significam o que parecem, entregues com a mais *sincera* das falsidades. Mal posso esperar para *não* te decepcionar. Ou talvez sim. Quem sabe? A ironia é tão... *clara*.

Pode começar com a sua *próxima* pergunta, que *com certeza* será de uma profundidade e originalid

In [26]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [27]:
from pydantic import BaseModel, Field

from typing import Literal, List, Dict


class TriagemOut(BaseModel):
    
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    
    campos_faltantes: List[str] = Field(default_factory=list)
 

In [28]:
llm_triagem = ChatGoogleGenerativeAI(
    
    model = 'gemini-2.5-flash',
    
    temperature = 0,
    
    api_key = api_key
)

In [31]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    
    saida: TriagemOut = triagem_chain.invoke([
        
        SystemMessage(content=TRIAGEM_PROMPT),
        
        HumanMessage(content=mensagem)
        
    ]
    )
    
    return saida.model_dump()
    


In [32]:
testes = ["Posso reembolsar a internet?", "Quero mais dias de trabalho remoto, como faço?", "posso reembolsar cursos ou treinamentos da Alura?", "Quantas capivaras tem no rio pinheiros"]

In [36]:
for mensagem_texto in testes:
    
    print(f"Pergunta: {mensagem_texto}\n -> Resposta: {triagem(mensagem_texto)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais dias de trabalho remoto, como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no rio pinheiros
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}

